In [1]:
from transformers import pipeline

C:\Users\lkkcp\anaconda3\envs\transformers\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [76]:
emo2idx = {}
idx2emo = {}
i = 0
with open('./data/emotions.txt','r') as f:
    for line in f:
        emo2idx[line.strip()]=i
        idx2emo[i] = line.strip()
        i+=1
emo_list = list(emo2idx.keys())
emo_list.remove('neutral')
#emo_list

In [5]:
train_df = pd.read_csv("data/train.tsv",sep='\t', header = None, names=['Text','Class','ID'])

In [268]:
def df_process(df):
    sample_size = df.shape[0]
    z = pd.DataFrame(np.zeros((sample_size,len(emo_list)),int),index=df.index, columns=emo_list)
    res_df = pd.concat([df,z],axis=1)
    #res_df.reset_index(drop=True,inplace=True)

    class_list = df['Class'].map(lambda x:x.split(','))#.tolist()
    for i in class_list.index:
        for c in class_list.loc[i]:
            res_df.loc[i,idx2emo[int(c)]]=1
        
    print('Dropped', res_df['neutral'].sum(), 'samples that are neutral')
        
    res_df.drop(res_df[res_df['neutral']==1].index,inplace=True)
    res_df.drop(['Class','neutral'],axis=1,inplace=True)
    res_df.reset_index(drop=True,inplace=True)
    return res_df

In [275]:
small_sample_size = 200
small_train_df = df_process(train_df[['Text','Class']].head(small_sample_size))
small_val_df = df_process(train_df.loc[small_sample_size:small_sample_size*2-1, ['Text','Class']])
small_test_df = df_process(train_df.loc[small_sample_size*2:small_sample_size*3-1, ['Text','Class']])

Dropped 76.0 samples that are neutral
Dropped 68.0 samples that are neutral
Dropped 62.0 samples that are neutral


In [276]:
small_train_df.head()

,Text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise
0,WHY THE FUCK IS BAYLESS ISOING,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,To make her feel threatened,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Dirty Southern Wankers,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,Yes I heard abt the f bombs! That has to be wh...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [277]:
small_train_df.to_csv('data/small_train.csv',index=False)
small_val_df.to_csv('data/small_val.csv',index=False)
small_test_df.to_csv('data/small_test.csv',index=False)


In [279]:
dataset = load_dataset("csv",data_files=
                       {"train": "data/small_train.csv",
                        "validation": "data/small_val.csv",
                         "test": "data/small_test.csv"})

Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 1003.02it/s]


Dataset csv downloaded and prepared to C:/Users/lkkcp/.cache/huggingface/datasets/csv/default-3db98f2b8151b599/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 370.22it/s]


In [280]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Text', 'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise'],
        num_rows: 124
    })
    validation: Dataset({
        features: ['Text', 'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise'],
        num_rows: 132
    })
    test: Dataset({
        features: ['Text', 'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disappro

In [225]:
model_checkpoint = 'bert-base-uncased'
MAXLEN = 128

In [176]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [312]:
def encode(samples):
    return tokenizer(samples['Text'], padding="max_length", truncation=True, max_length=MAXLEN)
def label_formatting(samples):
    labels_matrix = np.zeros((len(samples['Text']), len(emo_list)))
    for i, emo in enumerate(emo_list):
        labels_matrix[:,i] = samples[emo]
    return {'labels':labels_matrix.tolist()}

temp = dataset.map(encode,batched=True)
encoded_dataset = temp.map(label_formatting,batched=True,remove_columns=dataset['train'].column_names)

Loading cached processed dataset at C:\Users\lkkcp\.cache\huggingface\datasets\csv\default-3db98f2b8151b599\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-9f2ddb17391f21b0.arrow
Loading cached processed dataset at C:\Users\lkkcp\.cache\huggingface\datasets\csv\default-3db98f2b8151b599\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-701394d76a1564e9.arrow
Loading cached processed dataset at C:\Users\lkkcp\.cache\huggingface\datasets\csv\default-3db98f2b8151b599\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-7ddd3fc03db9c072.arrow


In [313]:
encoded_dataset.set_format("torch")

In [291]:
idx2emo_no_neutral = idx2emo.copy()
emo2idx_no_neutral = emo2idx.copy()
del idx2emo_no_neutral[27]
del emo2idx_no_neutral['neutral']

In [302]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased', num_labels=len(emo_list), problem_type="multi_label_classification",
    id2label=idx2emo_no_neutral, label2id=emo2idx_no_neutral
    )


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [294]:
batch_size = 8
metric_name = "f1"

In [306]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

In [298]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [314]:
encoded_dataset['train'][0]['labels']

tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [300]:
encoded_dataset['train'][0]['input_ids']

tensor([  101,  2339,  1996,  6616,  2003,  3016,  3238, 11163,  2075,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [315]:
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.7079, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.0056,  0.4173,  0.6280, -0.0743,  0.1634,  0.1295, -0.6982,  0.2129,
          0.3269,  0.2269, -0.1872,  0.0512,  0.3449,  0.1009, -0.0137,  0.1944,
         -0.2084, -0.2642,  0.0546, -0.4121, -0.6768, -0.5645,  0.5141,  0.1986,
          0.2496,  0.1539,  0.4166]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [316]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [317]:
trainer.train()

C:\Users\lkkcp\anaconda3\envs\transformers\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.620903,0.059979,0.477740,0.000000
2,No log,0.582016,0.036125,0.470312,0.000000


TrainOutput(global_step=32, training_loss=0.6377503275871277, metrics={'train_runtime': 8044.2154, 'train_samples_per_second': 0.031, 'train_steps_per_second': 0.004, 'total_flos': 16316547102720.0, 'train_loss': 0.6377503275871277, 'epoch': 2.0})

In [318]:
trainer.evaluate()

{'eval_loss': 0.620903491973877,
 'eval_f1': 0.059979317476732165,
 'eval_roc_auc': 0.4777397228261784,
 'eval_accuracy': 0.0,
 'eval_runtime': 1468.2067,
 'eval_samples_per_second': 0.09,
 'eval_steps_per_second': 0.012,
 'epoch': 2.0}